<a href="https://colab.research.google.com/github/SandeshRangreji/Scientific-Document-Summarization/blob/main/PEGASUS_XSUM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentencepiece
!pip install transformers
!pip install datasets
!pip install rouge_score
!pip install sacrebleu
!pip install bert_score

     |████████████████████████████████| 1.2 MB 5.1 MB/s 
     |████████████████████████████████| 3.1 MB 5.0 MB/s 
     |████████████████████████████████| 596 kB 84.2 MB/s 
     |████████████████████████████████| 895 kB 69.2 MB/s 
     |████████████████████████████████| 59 kB 8.2 MB/s 
     |████████████████████████████████| 3.3 MB 65.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 290 kB 5.0 MB/s 
     |████████████████████████████████| 243 kB 66.5 MB/s 
     |████████████████████████████████| 132 kB 103.4 MB/s 
     |████████████████████████████████| 1.1 MB 72.5 MB/s 
     |████████████████████████████████| 160 kB 88.0 MB/s 
     |████████████████████████████████| 192 kB 77.8 MB/s 
     |████████████████████████████████| 271 kB 75.3 MB/s 
     |████████████████████████████████| 90 kB 4.2 MB/s 
     |████████████████████████████████| 59 kB

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import torch
import numpy as np
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer, T5ForConditionalGeneration, T5TokenizerFast
from transformers import TFTrainer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import xml.etree.ElementTree as ET
from google.colab import drive
import os
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
os.mkdir("/content/PGN")
os.mkdir("/content/PGN/data")

In [ ]:
!unzip "/content/drive/MyDrive/Pointer Generator Networks/Data/scisumm.zip" -d "/content/PGN/data/"

In [ ]:
path = "/content/PGN/data/scisummnet_release1.1__20190413/top1000_complete/"
dir_list = os.listdir(path)
print(len(dir_list))

1009


In [ ]:
def get_text(xmlfile):
 sections = []
 intro_flag = False
 conclusion_flag = False
 tree = ET.parse(xmlfile)
 root = tree.getroot()
 for child in root:
  text = ""
  if child.tag != "S":
   if (child.attrib.get("title", child.tag).find("Introduction") != -1) and not intro_flag:
    intro_flag = True
    for grandchild in child:
     text = text + " " + grandchild.text
   if (child.attrib.get("title", child.tag).find("Conclusion") != -1) and not conclusion_flag:
    conclusion_flag = True
    for grandchild in child:
     text = text + " " + grandchild.text
  if text != "":
   sections.append(text)
 return sections, (intro_flag and conclusion_flag)

In [ ]:
a, b = get_text("/content/PGN/data/scisummnet_release1.1__20190413/top1000_complete/A00-1031/Documents_xml/A00-1031.xml") 
print((len(a)))  

2


In [ ]:
def get_articles(dir_list):
  articles = []
  blacklist = []
  articles_index = []
  total = 0
  for file_name in dir_list:
    if file_name not in blacklist:
      xmlfile = "/content/PGN/data/scisummnet_release1.1__20190413/top1000_complete/" + file_name + "/Documents_xml/" + file_name + ".xml"
      try:
        sections, flag = get_text(xmlfile)
        if flag == False:
          total+=1
        else:
          articles.append(sections)
          articles_index.append(xmlfile)
      except:
        blacklist.append(file_name)
  return articles, total, articles_index

In [ ]:
def word_count(sent):
  return len(sent.split(" "))

In [ ]:
articles, total, articles_index = get_articles(dir_list)

In [ ]:
def get_reference_summaries(articles_index):
  reference_summaries = []
  for file_name in articles_index:
    start_index = file_name.find("top1000_complete/") + 17
    end_index = file_name.find("/Documents_xml")
    file_name = file_name[start_index:end_index]
    ref_sum_file = "/content/PGN/data/scisummnet_release1.1__20190413/top1000_complete/" + file_name + "/summary/" + file_name + ".gold.txt"
    reference_summary = open(ref_sum_file, "r")
    reference_summaries.append(reference_summary.read().replace("\n", " ").replace("  "," ") + " </s>")
  return reference_summaries

In [ ]:
reference_summaries = get_reference_summaries(articles_index)

In [ ]:
input_sequences = []
for article in articles:
  # input_sequences.append("summarize:" + " ".join(article) + " </s>")
  input_sequences.append(" ".join(article))

In [ ]:
average_length = 0
input_lengths = []
for inp in input_sequences:
  input_lengths.append(word_count(inp))
print(sum(input_lengths)/len(input_sequences))

685.726148409894


In [ ]:
output_lengths = []
for out in reference_summaries:
  output_lengths.append(word_count(out))
print(sum(output_lengths)/len(output_lengths))

150.4876325088339


In [ ]:
max(output_lengths)

314

In [ ]:
from datasets import Dataset, DatasetDict

In [ ]:
shuffle_dataset = {"input" : input_sequences, "labels" : reference_summaries}

In [ ]:
shuffle_dataset = Dataset.from_dict(shuffle_dataset)
shuffle_dataset.shuffle(seed=42)

Dataset({
    features: ['input', 'labels'],
    num_rows: 566
})

In [ ]:
# train_dataset_dict = {"input" : input_sequences[:450], "labels" : reference_summaries[:450]}
# eval_dataset_dict = {"input" : input_sequences[450:500], "labels" : reference_summaries[450:500]}
# test_dataset_dict = {"input" : input_sequences[500:], "labels" : reference_summaries[500:]}
train_dataset_dict = shuffle_dataset[:450]
eval_dataset_dict = shuffle_dataset[450:500]
test_dataset_dict = shuffle_dataset[500:]

In [ ]:
train_dataset = Dataset.from_dict(train_dataset_dict)
eval_dataset = Dataset.from_dict(eval_dataset_dict)
test_dataset = Dataset.from_dict(test_dataset_dict)

In [ ]:
train_dataset[0]

{'input': " Given a word, its context and its possible meanings, the problem of word sense disambiguation (WSD) is to determine the meaning of the word in that context. WSD is useful in many natural language tasks, such as choosing the correct word in machine translation and coreference resolution. In several recent proposals (Hearst, 1991; Bruce and Wiebe, 1994; Leacock, Towwell, and Voorhees, 1996; Ng and Lee, 1996; Yarowsky, 1992; Yarowsky, 1994), statistical and machine learning techniques were used to extract classifiers from hand-tagged corpus. Yarowsky (Yarowsky, 1995) proposed an unsupervised method that used heuristics to obtain seed classifications and expanded the results to the other parts of the corpus, thus avoided the need to hand-annotate any examples. Most previous corpus-based WSD algorithms determine the meanings of polysemous words by exploiting their local contexts. A basic intuition that underlies those algorithms is the following: (1) Two occurrences of the same 

In [ ]:
dataset = DatasetDict({
    'train' : train_dataset,
    'validation': eval_dataset,
    'test': test_dataset,
})

In [ ]:
max_input_length = 512 # 1536
max_target_length = 162 # 162
source = "input"
target = "labels"

In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum')
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.36M [00:00<?, ?B/s]

In [ ]:
# from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

# model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
# tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

In [ ]:
def preprocess_function(data):
  input = data[source]
  targets = data[target]
  model_inputs = tokenizer(input, max_length=max_input_length, truncation=True)
  labels = None
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:
temp = preprocess_function(dataset['train'][:2])

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
batch_size = 1
source_lang = "article"
target_lang = "summary"
model_name = 'pegasus'
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=12,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    generation_max_length = 154
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from datasets import load_metric
metric = load_metric("rouge")
BertScore = load_metric("bertscore")

Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.93k [00:00<?, ?B/s]

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def convert_rouge_to_dict(score):
  
  rouge1 = score["rouge1"]
  rouge2 = score["rouge2"]
  rougeL = score["rougeL"]
  return {
      "rouge1_precision": rouge1.mid.precision,
      "rouge1_recall": rouge1.mid.recall,
      "rouge1_fmeasure": rouge1.mid.fmeasure,
      "rouge2_precision": rouge2.mid.precision,
      "rouge2_recall": rouge2.mid.recall,
      "rouge2_fmeasure": rouge2.mid.fmeasure,
      "rougeL_precision": rougeL.mid.precision,
      "rougeL_recall": rougeL.mid.recall,
      "rougeL_fmeasure": rougeL.mid.fmeasure,
  }

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    print(result)
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Using amp fp16 backend


In [ ]:
def generate_summaries(num_beams=10, num_return_sequences=10):
  torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
  generated_summaries = []
  for article in test_dataset["input"]:
    batch = tokenizer([article],truncation=True,padding='longest',max_length=max_input_length, return_tensors="pt").to(torch_device)
    translated = model.generate(**batch,max_length=max_target_length,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    generated_summaries.append(tgt_text[0])
  return generated_summaries

In [ ]:
def get_BertScore(BertScore, generated_summaries, test_dataset):
  predictions = generated_summaries
  references = test_dataset["labels"]
  results = BertScore.compute(predictions=predictions, references=references, lang="en")
  f1_score = [round(v, 2) for v in results["f1"]]
  precision = [round(v, 2) for v in results["precision"]]
  recall = [round(v, 2) for v in results["recall"]]
  return f1_score, precision, recall

In [ ]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])
# recall: n-grams in the reference summary are present in the generated summary
# precision: n-grams in the generated summary are present in the reference summary

The following columns in the evaluation set  don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: input.
***** Running Evaluation *****
  Num examples = 66
  Batch size = 1


{'rouge1': AggregateScore(low=Score(precision=0.6300217915238983, recall=0.09389241519814719, fmeasure=0.16126669274546157), mid=Score(precision=0.6577828460473214, recall=0.10572640485184973, fmeasure=0.17796452696344972), high=Score(precision=0.692119270851357, recall=0.11822436012218415, fmeasure=0.1949453509435764)), 'rouge2': AggregateScore(low=Score(precision=0.1961432816024111, recall=0.028722106316916445, fmeasure=0.04927622575680122), mid=Score(precision=0.2376286372787379, recall=0.03673789568820344, fmeasure=0.061950183089380384), high=Score(precision=0.27975684461908423, recall=0.04656957636966152, fmeasure=0.07661357480539045)), 'rougeL': AggregateScore(low=Score(precision=0.4467863428480407, recall=0.06710946534649019, fmeasure=0.11488663151773944), mid=Score(precision=0.4743629294696681, recall=0.07485685662213457, fmeasure=0.12629792119367445), high=Score(precision=0.5043648600405107, recall=0.08536262777672317, fmeasure=0.14036578853589032)), 'rougeLsum': AggregateScor

{'eval_gen_len': 25.1515,
 'eval_loss': 3.605175018310547,
 'eval_rouge1': 17.7965,
 'eval_rouge2': 6.195,
 'eval_rougeL': 12.6298,
 'eval_rougeLsum': 15.6818,
 'eval_runtime': 57.0336,
 'eval_samples_per_second': 1.157,
 'eval_steps_per_second': 1.157}

In [ ]:
generated_summaries = generate_summaries()
f1_score, precision, recall = get_BertScore(BertScore, generated_summaries, test_dataset)
print("F1 score : ", sum(f1_score)/len(f1_score))
print("Precision : ", sum(precision)/len(precision))
print("Recall : ", sum(recall)/len(recall))

Could not locate the tokenizer configuration file, will try to use the model config instead.
https://huggingface.co/roberta-large/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpke4ochdu


Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

storing https://huggingface.co/roberta-large/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/dea67b44b38d504f2523f3ddb6acb601b23d67bee52c942da336fa1283100990.94cae8b3a8dbab1d59b9d4827f7ce79e73124efa6bb970412cd503383a95f373
creating metadata file for /root/.cache/huggingface/transformers/dea67b44b38d504f2523f3ddb6acb601b23d67bee52c942da336fa1283100990.94cae8b3a8dbab1d59b9d4827f7ce79e73124efa6bb970412cd503383a95f373
loading configuration file https://huggingface.co/roberta-large/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/dea67b44b38d504f2523f3ddb6acb601b23d67bee52c942da336fa1283100990.94cae8b3a8dbab1d59b9d4827f7ce79e73124efa6bb970412cd503383a95f373
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_r

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

storing https://huggingface.co/roberta-large/resolve/main/vocab.json in cache at /root/.cache/huggingface/transformers/7c1ba2435b05451bc3b4da073c8dec9630b22024a65f6c41053caccf2880eb8f.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
creating metadata file for /root/.cache/huggingface/transformers/7c1ba2435b05451bc3b4da073c8dec9630b22024a65f6c41053caccf2880eb8f.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
https://huggingface.co/roberta-large/resolve/main/merges.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpfeoma8a9


...

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

storing https://huggingface.co/roberta-large/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/e16a2590deb9e6d73711d6e05bf27d832fa8c1162d807222e043ca650a556964.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730
creating metadata file for /root/.cache/huggingface/transformers/e16a2590deb9e6d73711d6e05bf27d832fa8c1162d807222e043ca650a556964.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730
loading file https://huggingface.co/roberta-large/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/7c1ba2435b05451bc3b4da073c8dec9630b22024a65f6c41053caccf2880eb8f.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/roberta-large/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/20b5a00a80e27ae9accbe25672aba42ad2d4d4cb2c4b9359b50ca8e34e107d6d.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/robe

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

storing https://huggingface.co/roberta-large/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/8e36ec2f5052bec1e79e139b84c2c3089cb647694ba0f4f634fec7b8258f7c89.c43841d8c5cd23c435408295164cda9525270aa42cd0cc9200911570c0342352
creating metadata file for /root/.cache/huggingface/transformers/8e36ec2f5052bec1e79e139b84c2c3089cb647694ba0f4f634fec7b8258f7c89.c43841d8c5cd23c435408295164cda9525270aa42cd0cc9200911570c0342352
loading weights file https://huggingface.co/roberta-large/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/8e36ec2f5052bec1e79e139b84c2c3089cb647694ba0f4f634fec7b8258f7c89.c43841d8c5cd23c435408295164cda9525270aa42cd0cc9200911570c0342352
All the weights of RobertaModel were initialized from the model checkpoint at roberta-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaModel for predictions without further training.


F1 score :  0.851060606060606
Precision :  0.8866666666666667
Recall :  0.8168181818181817


In [ ]:
generated_summaries[2]

'The aim of this work is to model phonetic errors in spelling correction algorithms.'

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: input.
***** Running training *****
  Num examples = 450
  Num Epochs = 12
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 5400
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1357: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,3.118346,36.157100,11.807600,23.412100,30.919000,118.780000
2,3.581200,3.050852,39.223300,12.788000,23.835500,34.314900,131.120000
3,3.166700,3.017421,40.229400,13.181300,23.194400,35.282700,136.540000
4,3.043500,2.996702,41.416100,13.703800,24.130700,36.350100,137.760000
5,2.904300,2.981251,41.550800,13.414000,24.237100,36.616900,140.940000
6,2.853700,2.975986,41.380000,14.042300,24.060800,36.563100,140.180000
7,2.782700,2.973202,41.524700,14.111700,24.022900,36.869100,139.600000
8,2.750600,2.966701,41.702300,13.655000,24.322000,36.980000,142.840000
9,2.685900,2.962931,42.698000,14.033000,24.738900,37.731200,142.160000
10,2.693900,2.966475,41.154700,13.541500,23.878800,36.618300,143.200000


The following columns in the evaluation set  don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: input.
***** Running Evaluation *****
  Num examples = 50
  Batch size = 1


{'rouge1': AggregateScore(low=Score(precision=0.39553925011040275, recall=0.3031966153340932, fmeasure=0.3339678959425549), mid=Score(precision=0.4465419231448513, recall=0.3285094690384006, fmeasure=0.36157138406759115), high=Score(precision=0.49565816352960146, recall=0.35198881249475783, fmeasure=0.38815997490212406)), 'rouge2': AggregateScore(low=Score(precision=0.12124183741231499, recall=0.08490996806215809, fmeasure=0.09626880525202645), mid=Score(precision=0.1541845274185551, recall=0.10309310642398889, fmeasure=0.11807600250831457), high=Score(precision=0.19039222156153116, recall=0.12212267780169596, fmeasure=0.14139639723574615)), 'rougeL': AggregateScore(low=Score(precision=0.2545410229406572, recall=0.19495760696872397, fmeasure=0.21412431365535742), mid=Score(precision=0.29068783592411096, recall=0.21244163028154833, fmeasure=0.23412116804857988), high=Score(precision=0.33091247234061155, recall=0.23035210735205913, fmeasure=0.2553324258662486)), 'rougeLsum': AggregateSco

Saving model checkpoint to pegasus-finetuned-article-to-summary/checkpoint-500
Configuration saved in pegasus-finetuned-article-to-summary/checkpoint-500/config.json
Model weights saved in pegasus-finetuned-article-to-summary/checkpoint-500/pytorch_model.bin
tokenizer config file saved in pegasus-finetuned-article-to-summary/checkpoint-500/tokenizer_config.json
Special tokens file saved in pegasus-finetuned-article-to-summary/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: input.
***** Running Evaluation *****
  Num examples = 50
  Batch size = 1


...

{'rouge1': AggregateScore(low=Score(precision=0.39607861342780504, recall=0.3888353574282503, fmeasure=0.3884755195543194), mid=Score(precision=0.4333698371779295, recall=0.4153476097994172, fmeasure=0.41630251175515565), high=Score(precision=0.4668242439258502, recall=0.43909696164464557, fmeasure=0.44232847217132953)), 'rouge2': AggregateScore(low=Score(precision=0.12072884363296116, recall=0.11497216022767624, fmeasure=0.11607903359477763), mid=Score(precision=0.14507468862518225, recall=0.13336034234989363, fmeasure=0.1365371233590158), high=Score(precision=0.17206825826899197, recall=0.15263028602288206, fmeasure=0.1581301924878474)), 'rougeL': AggregateScore(low=Score(precision=0.2263627582694634, recall=0.2220091335297891, fmeasure=0.22205807511777315), mid=Score(precision=0.25034034394444915, recall=0.23839009987599297, fmeasure=0.23947517980911154), high=Score(precision=0.27647415347579757, recall=0.2554668035977078, fmeasure=0.258908567644429)), 'rougeLsum': AggregateScore(lo

Saving model checkpoint to pegasus-finetuned-article-to-summary/checkpoint-5000
Configuration saved in pegasus-finetuned-article-to-summary/checkpoint-5000/config.json
Model weights saved in pegasus-finetuned-article-to-summary/checkpoint-5000/pytorch_model.bin
tokenizer config file saved in pegasus-finetuned-article-to-summary/checkpoint-5000/tokenizer_config.json
Special tokens file saved in pegasus-finetuned-article-to-summary/checkpoint-5000/special_tokens_map.json
Deleting older checkpoint [pegasus-finetuned-article-to-summary/checkpoint-3500] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: input.
***** Running Evaluation *****
  Num examples = 50
  Batch size = 1


{'rouge1': AggregateScore(low=Score(precision=0.40234496929395547, recall=0.3953969448492122, fmeasure=0.39476269831724664), mid=Score(precision=0.44074733951186085, recall=0.42158471582692647, fmeasure=0.42329962653432684), high=Score(precision=0.4749512093837183, recall=0.4458380494669796, fmeasure=0.4489005201588297)), 'rouge2': AggregateScore(low=Score(precision=0.12327004462193183, recall=0.1174248095366064, fmeasure=0.11847547558582847), mid=Score(precision=0.14847786758508835, recall=0.13664195019539205, fmeasure=0.1396129487235001), high=Score(precision=0.17533763662021928, recall=0.1569496993351841, fmeasure=0.162389539971795)), 'rougeL': AggregateScore(low=Score(precision=0.23038003538710924, recall=0.2243675683668517, fmeasure=0.22520595308970418), mid=Score(precision=0.25414577660418, recall=0.2413854223387066, fmeasure=0.2432113401878591), high=Score(precision=0.2798046471832306, recall=0.25941657141581836, fmeasure=0.2621147839592227)), 'rougeLsum': AggregateScore(low=Sco



Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=5400, training_loss=2.8932912416811343, metrics={'train_runtime': 4812.0194, 'train_samples_per_second': 1.122, 'train_steps_per_second': 1.122, 'total_flos': 9885015778590720.0, 'train_loss': 2.8932912416811343, 'epoch': 12.0})

In [ ]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

The following columns in the evaluation set  don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: input.
***** Running Evaluation *****
  Num examples = 66
  Batch size = 1


{'rouge1': AggregateScore(low=Score(precision=0.4467319790989197, recall=0.39377222830168007, fmeasure=0.41025635653410386), mid=Score(precision=0.47070098677084427, recall=0.41990094650458865, fmeasure=0.43398827601378837), high=Score(precision=0.49797468910256604, recall=0.44609183649489365, fmeasure=0.45893846327112303)), 'rouge2': AggregateScore(low=Score(precision=0.1458442582119903, recall=0.1321335074962515, fmeasure=0.13751364931458254), mid=Score(precision=0.16953317108641797, recall=0.155814396937112, fmeasure=0.1600656855333829), high=Score(precision=0.19613999818886294, recall=0.18276547376530505, fmeasure=0.18555067650188756)), 'rougeL': AggregateScore(low=Score(precision=0.26041543947205575, recall=0.22825814197339078, fmeasure=0.23861102923709954), mid=Score(precision=0.2844223869733678, recall=0.2501282542919625, fmeasure=0.258475242098131), high=Score(precision=0.30981020024926526, recall=0.2735880564955635, fmeasure=0.2807483024467139)), 'rougeLsum': AggregateScore(lo

{'epoch': 12.0,
 'eval_gen_len': 141.2424,
 'eval_loss': 2.796999931335449,
 'eval_rouge1': 43.3988,
 'eval_rouge2': 16.0066,
 'eval_rougeL': 25.8475,
 'eval_rougeLsum': 38.5728,
 'eval_runtime': 212.7968,
 'eval_samples_per_second': 0.31,
 'eval_steps_per_second': 0.31}

In [ ]:
generated_summaries = generate_summaries()
f1_score, precision, recall = get_BertScore(BertScore, generated_summaries, test_dataset)
print("F1 score : ", sum(f1_score)/len(f1_score))
print("Precision : ", sum(precision)/len(precision))
print("Recall : ", sum(recall)/len(recall))

F1 score :  0.8537878787878788
Precision :  0.8583333333333334
Recall :  0.8489393939393942


In [ ]:
# before training
# F1 score :  0.8512121212121213
# Precision :  0.8710606060606059
# Recall :  0.8319696969696969

# temp = 1.5
# F1 score :  0.8556060606060607
# Precision :  0.8637878787878792
# Recall :  0.8466666666666669

# temp = 2.0
# F1 score :  0.8556060606060607
# Precision :  0.8637878787878792
# Recall :  0.8466666666666669

In [ ]:
generated_summaries[2]

'Two Different Error Models for Spelling and Typographic Errors We present two different error models for spelling and phonetic errors. One of them is a letter-based error model which is exactly the Brill and Moore model trained on a similar dataset. The other is a phone-sequence-to-phone-sequence error model trained on the same data as the first model, but using the pronunciations of the correct words and estimated pronunciations of the misspellings to learn phone-sequence-to-phone-sequence edits and estimate their probabilities.'

In [ ]:
test_dataset["labels"][2]

'Pronunciation Modeling For Improved Spelling Correction This paper presents a method for incorporating word pronunciation information in a noisy channel model for spelling correction. The proposed method builds an explicit error model for word pronunciations. By modeling pronunciation similarities between words we achieve a substantial performance improvement over the previous best performing models for spelling correction. We consider a pronunciation variation model to generate multiple pronunciations for each canonical pronunciation in a pronouncing dictionary. We extend Brill and Moore (2000) to consider edits over both letter sequences and sequences of phones in the pronunciations of the word and misspelling. We use the noisy channel model approach to determine the types and weights of edit operations. Since a spelling correction model needs to rank candidate words rather than candidate pronunciations, we derive an error model that determines the probability that a word w was spel